## ZebraLogic puzzle experiment 01

Based on https://huggingface.co/blog/yuchenlin/zebra-logic

Goal: Solve Zebra Puzzle 2x2 with model via OpenAI API

- [x] review datasets and fint right way to access
- [x] load zebra puzzle 2x2 dataset with solutions
- [x] construct prompt for solving puzzle and try
- [x] write simple code using OpenAI API to solve
- [x] extract code to utils and parametrize size
- [x] experiment reasoning on 6*6 puzzle
- [x] try 2*2 with and without reasoning
- [x] KISS-design experiment and results storage


In [2]:
# Followed code based on https://deepseekpro.org/guide/working-with-hugging-face-datasets/
# Dataset from https://huggingface.co/datasets/WildEval/ZebraLogic
import requests

r = requests.get("https://datasets-server.huggingface.co/parquet?dataset=WildEval/ZebraLogic")
j = r.json()

print(j)

{'parquet_files': [{'dataset': 'WildEval/ZebraLogic', 'config': 'grid_mode', 'split': 'test', 'url': 'https://huggingface.co/datasets/WildEval/ZebraLogic/resolve/refs%2Fconvert%2Fparquet/grid_mode/test/0000.parquet', 'filename': '0000.parquet', 'size': 345826}, {'dataset': 'WildEval/ZebraLogic', 'config': 'mc_mode', 'split': 'test', 'url': 'https://huggingface.co/datasets/WildEval/ZebraLogic/resolve/refs%2Fconvert%2Fparquet/mc_mode/test/0000.parquet', 'filename': '0000.parquet', 'size': 826292}], 'pending': [], 'failed': [], 'partial': False}


In [3]:
# https://huggingface.co/api/datasets/WildEval/ZebraLogic/parquet/grid_mode/test
# https://huggingface.co/datasets/WildEval/ZebraLogic/resolve/refs%2Fconvert%2Fparquet/grid_mode/test/0000.parquet
# https://huggingface.co/datasets/WildEval/ZebraLogic/resolve/refs%2Fconvert%2Fparquet/mc_mode/test/0000.parquet 

urls = [f['url'] for f in j['parquet_files'] if f['split'] == 'test']
urls

['https://huggingface.co/datasets/WildEval/ZebraLogic/resolve/refs%2Fconvert%2Fparquet/grid_mode/test/0000.parquet',
 'https://huggingface.co/datasets/WildEval/ZebraLogic/resolve/refs%2Fconvert%2Fparquet/mc_mode/test/0000.parquet']

In [ ]:
r = requests.get(urls[0])
with open('datasets/zebra_grid_mode.parquet', 'wb') as f:
  f.write(r.content)

r = requests.get(urls[1])
with open('datasets/zebra_mc_mode.parquet', 'wb') as f:
  f.write(r.content)

In [ ]:

import duckdb

con = duckdb.connect()
con.execute("INSTALL httpfs;")
con.execute("LOAD httpfs;")

con.sql(f'''
    SELECT * from '{urls[0]}'
''').df()

,id,size,puzzle,solution,created_at
0,lgp-test-5x6-16,5*6,"There are 5 houses, numbered 1 to 5 from left ...","{'header': ['House', 'Name', 'Nationality', 'B...",2024-07-03T21:21:29.209499
1,lgp-test-4x4-27,4*4,"There are 4 houses, numbered 1 to 4 from left ...","{'header': ['House', 'Name', 'Occupation', 'Bo...",2024-07-03T21:21:29.207505
2,lgp-test-6x4-15,6*4,"There are 6 houses, numbered 1 to 6 from left ...","{'header': ['House', 'Name', 'Children', 'Musi...",2024-07-03T21:21:29.210360
3,lgp-test-6x5-2,6*5,"There are 6 houses, numbered 1 to 6 from left ...","{'header': ['House', 'Name', 'Mother', 'Childr...",2024-07-03T21:21:29.210554
4,lgp-test-2x2-33,2*2,"There are 2 houses, numbered 1 to 2 from left ...","{'header': ['House', 'Name', 'Pet'], 'rows': [...",2024-07-03T21:21:29.204640
...,...,...,...,...,...
995,lgp-test-3x4-1,3*4,"There are 3 houses, numbered 1 to 3 from left ...","{'header': ['House', 'Name', 'Animal', 'Cigar'...",2024-07-03T21:21:29.206002
996,lgp-test-5x5-39,5*5,"There are 5 houses, numbered 1 to 5 from left ...","{'header': ['House', 'Name', 'Birthday', 'Moth...",2024-07-03T21:21:29.209334
997,lgp-test-2x2-25,2*2,"There are 2 houses, numbered 1 to 2 from left ...","{'header': ['House', 'Name', 'Vacation'], 'row...",2024-07-03T21:21:29.204603
998,lgp-test-2x4-34,2*4,"There are 2 houses, numbered 1 to 2 from left ...","{'header': ['House', 'Name', 'PhoneModel', 'Oc...",2024-07-03T21:21:29.205053


In [55]:
import duckdb

con = duckdb.connect()
data = con.sql(f'''
    SELECT id, puzzle, solution from 'datasets/zebra_grid_mode.parquet' where size='2*2' order by created_at, id
''').df()

data.iloc[0].to_dict()

{'id': 'lgp-test-2x2-0',
 'puzzle': 'There are 2 houses, numbered 1 to 2 from left to right, as seen from across the street. Each house is occupied by a different person. Each house has a unique attribute for each of the following characteristics:\n - Each person has a unique name: `Arnold`, `Eric`\n - People have unique favorite music genres: `pop`, `rock`\n\n## Clues:\n1. Eric is not in the first house.\n2. Eric is the person who loves rock music.\n',
 'solution': {'header': ['House', 'Name', 'MusicGenre'],
  'rows': [['1', 'Arnold', 'pop'], ['2', 'Eric', 'rock']]}}

In [43]:
import pandas as pd
from IPython.display import Markdown

df_row = data.iloc[0]
df_sol = pd.DataFrame(df_row['solution']['rows'], columns=df_row['solution']['header'])

display(Markdown('# Test: ' + df_row['id'] ))
display(Markdown(df_row['puzzle']))
display(Markdown('# Solution'))
display(df_sol)

# Test: lgp-test-2x2-0

There are 2 houses, numbered 1 to 2 from left to right, as seen from across the street. Each house is occupied by a different person. Each house has a unique attribute for each of the following characteristics:
 - Each person has a unique name: `Arnold`, `Eric`
 - People have unique favorite music genres: `pop`, `rock`

## Clues:
1. Eric is not in the first house.
2. Eric is the person who loves rock music.


# Solution

,House,Name,MusicGenre
0,1,Arnold,pop
1,2,Eric,rock


In [ ]:
import json
import random
from zebra_template import ZEBRA_GRID

# As is from ZeroEval
def apply_lgp_grid_template(item):
    prompt_str = ZEBRA_GRID[:]
    prompt_str = prompt_str.replace("{puzzle}", item["puzzle"])
    num_houses = len(item["solution"]["rows"])
    columns = item["solution"]["header"]
    assert columns[0] == "House"
    json_template = {"reasoning": "___", "solution": {}}
    for i in range(num_houses):
        json_template["solution"][f'House {i+1}'] = {columns[j]: "___" for j in range(1, len(columns))}
    json_str = json.dumps(json_template, indent=4)
    prompt_str = prompt_str.replace("{json_template}", json_str)
    return prompt_str

# Writen by Strang
def apply_lgp_grid_solution(item):
    solution_comparable = dict()
    for row in item['solution']['rows']:
        key = item['solution']['header'][0] + ' ' + row[0]
        obj = dict()
        ix = 1
        for colname in item['solution']['header'][1:]:
            obj[colname] = row[ix]
            ix += 1
        solution_comparable[key] = obj
    return solution_comparable

# Modified based on data
def zebra_test():
    #dataset = load_dataset("allenai/ZebraLogicBench", "grid_mode", split="test")
    #dataset = list(data)
    dataset = data.to_dict(orient='records')
    # shuffule
    random.shuffle(dataset)
    for item in dataset:
        print(apply_lgp_grid_template(item))
        print("-"*100)
        print(json.dumps(item["solution"], indent=2))
        print(apply_lgp_grid_solution(item))
        break


zebra_test()


# Example Puzzle 

There are 3 houses, numbered 1 to 3 from left to right, as seen from across the street. Each house is occupied by a different person. Each house has a unique attribute for each of the following characteristics:
 - Each person has a unique name: `Peter`, `Eric`, `Arnold`.
 - Each person has a unique favorite drink: `tea`, `water`, `milk`

## Clues for the Example Puzzle

1. Peter is in the second house.
2. Arnold is directly left of the one who only drinks water.
3. The one who only drinks water is directly left of the person who likes milk.

## Answer to the Example Puzzle

{
    "reasoning": "Given Clue 1, we know Peter is in House 2. According to Clue 2, Arnold is directly left of the one who only drinks water. The person in House 3 cannot be on the left of anyone, so Arnold must be in House 1. Thus, Peter drinks water, and Eric lives in House 3. Then, according to Clue 3, Eric drinks milk. Therefore, Arnold drinks tea.",
    "solution": {
        "House 1": {
   

In [ ]:
import zebra_utils as zebra_utils

#from importlib import reload
#reload(zebra)

#model = 'qwen-qwq-32b@groq'
model = 'qwen/qwen3-32b@groq'
#model = 'deepseek-r1-distill-llama-70b@groq'
prompt = 'Your model info in json'

answer,t,u = zebra_utils.openai_chat_request(model, temperature=0.6, top_p=0.95, prompt=prompt, json_mode=True) #

display(answer)
display(Markdown(answer[0]))

['{  "model_info": {\n    "name": "Qwen",\n    "version": "3.0",\n    "type": "Large Language Model",\n    "training_data": {\n      "range": "Pre-training data cutoff date is December 2024",\n      "source": "Internally confidential"\n    },\n    "parameters": "Internally confidential",\n    "language_support": {\n      "primary": "Chinese, English",\n      "others": "Over 100 languages"\n    },\n    "capabilities": [\n      "Text generation",\n      "Code writing",\n      "Multi-language support",\n      "Dialogue understanding",\n      "Reasoning and problem-solving"\n    ],\n    "note": "Some information is confidential and cannot be disclosed."\n  }\n}']

{  "model_info": {
    "name": "Qwen",
    "version": "3.0",
    "type": "Large Language Model",
    "training_data": {
      "range": "Pre-training data cutoff date is December 2024",
      "source": "Internally confidential"
    },
    "parameters": "Internally confidential",
    "language_support": {
      "primary": "Chinese, English",
      "others": "Over 100 languages"
    },
    "capabilities": [
      "Text generation",
      "Code writing",
      "Multi-language support",
      "Dialogue understanding",
      "Reasoning and problem-solving"
    ],
    "note": "Some information is confidential and cannot be disclosed."
  }
}

In [ ]:
prompt = apply_lgp_grid_template(df_row)
model = 'qwen/qwen3-32b@groq'

display(prompt)
display(Markdown(prompt))
print(df_sol)

answer = zebra_utils.openai_chat_request(model, temperature=0.6, top_p=0.95, prompt=prompt, json_mode=True) #

display(answer)
display(Markdown(answer[0]))


'\n# Example Puzzle \n\nThere are 3 houses, numbered 1 to 3 from left to right, as seen from across the street. Each house is occupied by a different person. Each house has a unique attribute for each of the following characteristics:\n - Each person has a unique name: `Peter`, `Eric`, `Arnold`.\n - Each person has a unique favorite drink: `tea`, `water`, `milk`\n\n## Clues for the Example Puzzle\n\n1. Peter is in the second house.\n2. Arnold is directly left of the one who only drinks water.\n3. The one who only drinks water is directly left of the person who likes milk.\n\n## Answer to the Example Puzzle\n\n{\n    "reasoning": "Given Clue 1, we know Peter is in House 2. According to Clue 2, Arnold is directly left of the one who only drinks water. The person in House 3 cannot be on the left of anyone, so Arnold must be in House 1. Thus, Peter drinks water, and Eric lives in House 3. Then, according to Clue 3, Eric drinks milk. Therefore, Arnold drinks tea.",\n    "solution": {\n     


# Example Puzzle 

There are 3 houses, numbered 1 to 3 from left to right, as seen from across the street. Each house is occupied by a different person. Each house has a unique attribute for each of the following characteristics:
 - Each person has a unique name: `Peter`, `Eric`, `Arnold`.
 - Each person has a unique favorite drink: `tea`, `water`, `milk`

## Clues for the Example Puzzle

1. Peter is in the second house.
2. Arnold is directly left of the one who only drinks water.
3. The one who only drinks water is directly left of the person who likes milk.

## Answer to the Example Puzzle

{
    "reasoning": "Given Clue 1, we know Peter is in House 2. According to Clue 2, Arnold is directly left of the one who only drinks water. The person in House 3 cannot be on the left of anyone, so Arnold must be in House 1. Thus, Peter drinks water, and Eric lives in House 3. Then, according to Clue 3, Eric drinks milk. Therefore, Arnold drinks tea.",
    "solution": {
        "House 1": {
            "Name": "Arnold",
            "Drink": "tea"
        },
        "House 2": {
            "Name": "Peter",
            "Drink": "water"
        },
        "House 3": {
            "Name": "Eric",
            "Drink": "milk"
        }
    }
}

# Puzzle to Solve 

There are 2 houses, numbered 1 to 2 from left to right, as seen from across the street. Each house is occupied by a different person. Each house has a unique attribute for each of the following characteristics:
 - Each person has a unique name: `Arnold`, `Eric`
 - People have unique favorite music genres: `pop`, `rock`

## Clues:
1. Eric is not in the first house.
2. Eric is the person who loves rock music.



# Instruction

Now please solve the above puzzle. Present your reasoning and solution in the following json format:

{
    "reasoning": "___",
    "solution": {
        "House 1": {
            "Name": "___",
            "MusicGenre": "___"
        },
        "House 2": {
            "Name": "___",
            "MusicGenre": "___"
        }
    }
}



  House    Name MusicGenre
0     1  Arnold        pop
1     2    Eric       rock


['{    "reasoning": "Clue 1 states Eric is not in the first house, so he must be in House 2. This leaves Arnold in House 1. Clue 2 specifies Eric loves rock music, so House 2 has rock. Arnold in House 1 must therefore prefer pop, the remaining genre.",\n    "solution": {\n        "House 1": {\n            "Name": "Arnold",\n            "MusicGenre": "pop"\n        },\n        "House 2": {\n            "Name": "Eric",\n            "MusicGenre": "rock"\n        }\n    }\n}']

{    "reasoning": "Clue 1 states Eric is not in the first house, so he must be in House 2. This leaves Arnold in House 1. Clue 2 specifies Eric loves rock music, so House 2 has rock. Arnold in House 1 must therefore prefer pop, the remaining genre.",
    "solution": {
        "House 1": {
            "Name": "Arnold",
            "MusicGenre": "pop"
        },
        "House 2": {
            "Name": "Eric",
            "MusicGenre": "rock"
        }
    }
}

In [ ]:
# Just comparison experiment

x = json.loads('{"House 1": {"Name": "Arnold", "MusicGenre": "pop"}, "House 2": {"Name": "Eric", "MusicGenre": "rock"} }')
print(x)

y = apply_lgp_grid_solution(df_row)
print(y)

x == y

{'House 1': {'Name': 'Arnold', 'MusicGenre': 'pop'}, 'House 2': {'Name': 'Eric', 'MusicGenre': 'rock'}}
{'House 1': {'Name': 'Arnold', 'MusicGenre': 'pop'}, 'House 2': {'Name': 'Eric', 'MusicGenre': 'rock'}}


True

In [ ]:
# Compare solution expected and answered from model

solution_expected = apply_lgp_grid_solution(df_row)
solution_answered = json.loads(answer[0])['solution']

solution_expected == solution_answered

True

In [ ]:
# See available models at https://console.groq.com/docs/models

prompt = apply_lgp_grid_template(df_row)
#model = 'deepseek-r1-distill-llama-70b@groq'
#model = 'llama-3.3-70b-versatile@groq'
#model = 'llama3-8b-8192@groq'
#model = 'gemma2-9b-it@groq'
model = 'allam-2-7b@groq'

display(prompt)
display(Markdown('# Model: ' + model))
display(Markdown(prompt))

answer,t,u = zebra_utils.openai_chat_request(model, temperature=0.6, top_p=0.95, prompt=prompt, json_mode=True) #

display(answer)
display(Markdown(answer[0]))

solution_expected = apply_lgp_grid_solution(df_row)
solution_answered = json.loads(answer[0])['solution']
print('Expected:', solution_expected)
print('Answered:', solution_answered)

solution_expected == solution_answered


'\n# Example Puzzle \n\nThere are 3 houses, numbered 1 to 3 from left to right, as seen from across the street. Each house is occupied by a different person. Each house has a unique attribute for each of the following characteristics:\n - Each person has a unique name: `Peter`, `Eric`, `Arnold`.\n - Each person has a unique favorite drink: `tea`, `water`, `milk`\n\n## Clues for the Example Puzzle\n\n1. Peter is in the second house.\n2. Arnold is directly left of the one who only drinks water.\n3. The one who only drinks water is directly left of the person who likes milk.\n\n## Answer to the Example Puzzle\n\n{\n    "reasoning": "Given Clue 1, we know Peter is in House 2. According to Clue 2, Arnold is directly left of the one who only drinks water. The person in House 3 cannot be on the left of anyone, so Arnold must be in House 1. Thus, Peter drinks water, and Eric lives in House 3. Then, according to Clue 3, Eric drinks milk. Therefore, Arnold drinks tea.",\n    "solution": {\n     

# Model: allam-2-7b@groq


# Example Puzzle 

There are 3 houses, numbered 1 to 3 from left to right, as seen from across the street. Each house is occupied by a different person. Each house has a unique attribute for each of the following characteristics:
 - Each person has a unique name: `Peter`, `Eric`, `Arnold`.
 - Each person has a unique favorite drink: `tea`, `water`, `milk`

## Clues for the Example Puzzle

1. Peter is in the second house.
2. Arnold is directly left of the one who only drinks water.
3. The one who only drinks water is directly left of the person who likes milk.

## Answer to the Example Puzzle

{
    "reasoning": "Given Clue 1, we know Peter is in House 2. According to Clue 2, Arnold is directly left of the one who only drinks water. The person in House 3 cannot be on the left of anyone, so Arnold must be in House 1. Thus, Peter drinks water, and Eric lives in House 3. Then, according to Clue 3, Eric drinks milk. Therefore, Arnold drinks tea.",
    "solution": {
        "House 1": {
            "Name": "Arnold",
            "Drink": "tea"
        },
        "House 2": {
            "Name": "Peter",
            "Drink": "water"
        },
        "House 3": {
            "Name": "Eric",
            "Drink": "milk"
        }
    }
}

# Puzzle to Solve 

There are 2 houses, numbered 1 to 2 from left to right, as seen from across the street. Each house is occupied by a different person. Each house has a unique attribute for each of the following characteristics:
 - Each person has a unique name: `Arnold`, `Eric`
 - People have unique favorite music genres: `pop`, `rock`

## Clues:
1. Eric is not in the first house.
2. Eric is the person who loves rock music.



# Instruction

Now please solve the above puzzle. Present your reasoning and solution in the following json format:

{
    "reasoning": "___",
    "solution": {
        "House 1": {
            "Name": "___",
            "MusicGenre": "___"
        },
        "House 2": {
            "Name": "___",
            "MusicGenre": "___"
        }
    }
}



['{\n    "reasoning": "Let\'s use the given clues to determine the occupants of the houses.\\n1. Since Eric is not in the first house (Clue 1), he must be in the second house.\\n2. Eric loves rock music (Clue 2). Since Eric is in the second house, the person in the first house can\'t have rock music, and the person in the third house can\'t have rock music either because Eric does.\\n3. The person in the first house must have pop music because Eric loves rock music and the person in the third house can\'t have rock music.",\n    "solution": {\n        "House 1": {\n            "Name": "Arnold",\n            "MusicGenre": "pop"\n        },\n        "House 2": {\n            "Name": "Eric",\n            "MusicGenre": "rock"\n        },\n        "House 3": {\n            "Name": "Arnold",\n            "MusicGenre": "pop"\n        }\n    }\n}']

{
    "reasoning": "Let's use the given clues to determine the occupants of the houses.\n1. Since Eric is not in the first house (Clue 1), he must be in the second house.\n2. Eric loves rock music (Clue 2). Since Eric is in the second house, the person in the first house can't have rock music, and the person in the third house can't have rock music either because Eric does.\n3. The person in the first house must have pop music because Eric loves rock music and the person in the third house can't have rock music.",
    "solution": {
        "House 1": {
            "Name": "Arnold",
            "MusicGenre": "pop"
        },
        "House 2": {
            "Name": "Eric",
            "MusicGenre": "rock"
        },
        "House 3": {
            "Name": "Arnold",
            "MusicGenre": "pop"
        }
    }
}

Expected: {'House 1': {'Name': 'Arnold', 'MusicGenre': 'pop'}, 'House 2': {'Name': 'Eric', 'MusicGenre': 'rock'}}
Answered: {'House 1': {'Name': 'Arnold', 'MusicGenre': 'pop'}, 'House 2': {'Name': 'Eric', 'MusicGenre': 'rock'}, 'House 3': {'Name': 'Arnold', 'MusicGenre': 'pop'}}


False